# CVE 分析智能体


这个例子主要的教学演示：
1. 如何导入外部知识，从 HTML、PDF、MARKDOWN、TEXT 等格式中导入
2. 如何将外部知识向量化、存储到向量数据库中，并编排 Agent 查询工具

另外我们还将学习:
- Arize Phoenix llm 开发观测平台，使用 Arize Phoenix 观测 Agent 的思维过程

这个例子将几个 **预定义的 CVE 知识** 导入到向量数据库中，并编排 Agent 查询工具，然后让 Agent 分析用户的问题，选择是否使用外部知识，并给出分析结果。

CVE  数据网站: [www.cvedetails.com](https://www.cvedetails.com)

In [23]:
# API Key 设置
import os

# os.environ["OPENAI_API_KEY"] = "sk-******"
# os.environ["OPENAI_API_BASE"] = "https://api.****"

# 安装依赖库

In [ ]:
%pip install llama-index
%pip install llama-index-llms-openai
%pip install llama-index-callbacks-arize-phoenix
%pip install tqdm

# CVE 分析智能体

## 非结构化数据解析
TXT 是常见的非结构化数据，我们可以使用 llama-index 封装的 `UnstructuredReader` 来解析 TXT 数据。

In [ ]:
from llama_index.readers.file import UnstructuredReader

data_dir = 'data/cve-agent'

reader = UnstructuredReader()
all_cve_docs = []
all_product_docs = []
for file_name in os.listdir(data_dir):
    if not file_name.endswith('.txt'):
        continue
    
    file_path = os.path.join(data_dir, file_name)
    documents = reader.load_data(file=file_path)
    if file_name.startswith('CVE-'):
        all_cve_docs.extend(documents)
    elif file_name.startswith('product-'):
        all_product_docs.extend(documents)

# 导入观测插件


In [55]:
from llama_index.core.settings import Settings
from llama_index.core.callbacks import CallbackManager

callback_manager = CallbackManager()
Settings.callback_manager = callback_manager

# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## 向量化

In [56]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.chunk_size = 512
Settings.chunk_overlap = 64
Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


### 新数据向量化

In [57]:
cve_storage_context = StorageContext.from_defaults()
product_storage_context = StorageContext.from_defaults()
cve_index = VectorStoreIndex.from_documents(all_cve_docs, storage_context=cve_storage_context, show_progress=True)
product_index = VectorStoreIndex.from_documents(all_product_docs, storage_context=product_storage_context, show_progress=True)
cve_storage_context.persist(persist_dir='data/cve-agent/storage/cve')
product_storage_context.persist(persist_dir='data/cve-agent/storage/product')

cve_query_engine = cve_index.as_query_engine()
product_query_engine = product_index.as_query_engine()

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

### 加载已向量化的数据

In [58]:
from llama_index.core import load_index_from_storage

cve_index_storage_context = StorageContext.from_defaults(persist_dir='data/cve-agent/storage/cve')
product_index_storage_context = StorageContext.from_defaults(persist_dir='data/cve-agent/storage/product')

cve_index = load_index_from_storage(cve_index_storage_context)
product_index = load_index_from_storage(product_index_storage_context)

cve_query_engine = cve_index.as_query_engine()
product_query_engine = product_index.as_query_engine()

In [59]:
response = cve_query_engine.query("Cursor 有哪些漏洞？CVE 编号是什么？")
response

Response(response='Cursor 软件中存在一个漏洞，其 CVE 编号是 CVE-2025-27554。这个漏洞允许远程攻击者通过在 package.json 中使用 postinstall 脚本在构建服务器上执行任意命令，并因此可以部署更新到任何应用程序。', source_nodes=[NodeWithScore(node=TextNode(id_='de994e4f-3609-4aae-9e3e-617e9a0f3c01', embedding=None, metadata={'last_modified': '2025-03-02T14:47:26', 'languages': '["eng"]', 'file_directory': 'data/cve-agent', 'filename': 'CVE-2025-27416.txt', 'filetype': 'text/plain'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='CVE-2025-27416.txt', node_type='4', metadata={'last_modified': '2025-03-02T14:47:26', 'languages': '["eng"]', 'file_directory': 'data/cve-agent', 'filename': 'CVE-2025-27416.txt', 'filetype': 'text/plain'}, hash='c79e97c4a822e329dbf5af256eb2f20ecb2e0b7a1206f1a8842f46cdca756b2d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='04ffb9b1-3f94-480b-85e5-553dc44ee780', node_type='1', metadata={'last_modified': '2025-03-02T14:47:26', 'langu

In [60]:
response = product_query_engine.query("Cursor 是什么？")
response

Response(response='Cursor 是一款智能化的代码编辑器，基于 VS Code 构建，旨在通过 AI 集成来提升开发者的工作效率。它提供了多种核心功能，如智能代码补全、代码查询对话界面、复杂代码修改支持等，帮助开发者更快速高效地完成编码任务。用户可以通过下载和安装 Cursor，在短时间内上手这些强大的 AI 功能。', source_nodes=[NodeWithScore(node=TextNode(id_='a58caba4-529f-4bff-b4fb-0bd93122e858', embedding=None, metadata={'last_modified': '2025-03-02T14:54:38', 'languages': '["eng"]', 'file_directory': 'data/cve-agent', 'filename': 'product-cursor.txt', 'filetype': 'text/plain'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='product-cursor.txt', node_type='4', metadata={'last_modified': '2025-03-02T14:54:38', 'languages': '["eng"]', 'file_directory': 'data/cve-agent', 'filename': 'product-cursor.txt', 'filetype': 'text/plain'}, hash='3a4701acfd7ae45f262eeb7f61917223286c095a40487dd30c9c33ead07f9ec9'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='0e9da1d8-20f0-48c8-ad7b-da4c2d3f2975', node_type='1', metadata={'last_modified'

# 编排 Agent 查询工具

In [61]:

from llama_index.core.tools import QueryEngineTool, ToolMetadata

cve_query_engine_tool = QueryEngineTool(
    query_engine=cve_query_engine,
    metadata=ToolMetadata(
        name="cve_query",
        description="查询 CVE 漏洞信息"
    )
)

product_query_engine_tool = QueryEngineTool(
    query_engine=product_query_engine,
    metadata=ToolMetadata(
        name="product_query",
        description="查询产品信息"
    )
)

# 构建 Agent

In [62]:
from llama_index.agent.openai import OpenAIAgent

tools = [cve_query_engine_tool, product_query_engine_tool]

agent = OpenAIAgent.from_tools(tools, verbose=True)
response = agent.query("CVE-2025-27554 是什么漏洞？这个漏洞将影响到哪些产品？对于影响到的产品需要简介，并提供修复方案。输出报告格式为 markdown 格式。")

Added user message to memory: CVE-2025-27554 是什么漏洞？这个漏洞将影响到哪些产品？对于影响到的产品需要简介，并提供修复方案。输出报告格式为 markdown 格式。
=== Calling Function ===
Calling function: cve_query with args: {"input":"CVE-2025-27554"}
Got output: CVE-2025-27554 describes a vulnerability in ToDesktop before October 3, 2024, which is used by Cursor and other applications. This vulnerability allows remote attackers to execute arbitrary commands on the build server, such as reading secrets from the desktopify config.prod.json file, and subsequently deploying updates to any app via a postinstall script in package.json. It is noted that no exploitation of this vulnerability has occurred.

=== Calling Function ===
Calling function: product_query with args: {"input": "ToDesktop"}
Got output: ToDesktop is a platform that allows you to transform your web app codebase into a cross-platform desktop application with native functionality. It supports integration with various web stacks and frameworks, enabling the creation of desktop ap

## 输出报告

In [63]:
from IPython.display import Markdown
display(Markdown(response.response))

# CVE-2025-27554 漏洞报告

## 漏洞描述
CVE-2025-27554 描述了 ToDesktop 在 2024 年 10 月 3 日之前的版本中的一个漏洞，该漏洞被 Cursor 和其他应用程序使用。此漏洞允许远程攻击者在构建服务器上执行任意命令，例如从 desktopify config.prod.json 文件中读取机密信息，并随后通过 package.json 中的 postinstall 脚本部署更新。需要注意的是，目前尚未发生对该漏洞的利用。

## 影响产品
### ToDesktop
ToDesktop 是一个平台，允许您将 Web 应用代码库转换为具有本机功能的跨平台桌面应用程序。它支持与各种 Web 堆栈和框架的集成，能够创建适用于 Mac、Windows 和 Linux 的桌面应用程序。使用 ToDesktop Builder，您可以快速将 Web 应用转换为桌面应用，添加本机功能，并使用系统级 API 自定义应用。

### Cursor
Cursor 是一个基于 VS Code 的 AI 驱动代码编辑器，旨在增强您的开发工作流程。它提供智能代码补全、代码查询聊天和复杂开发任务的助手等功能。您可以轻松地从其网站下载并安装 Cursor，并支持一键导入所有 VS Code 扩展和设置。Cursor 提供 14 天的 Pro 计划免费试用，无需信用卡。该编辑器是可自定义的，允许调整 AI 功能和标准编辑器设置。

## 修复方案
1. **更新 ToDesktop**：确保使用 ToDesktop 的最新版本，该版本已修复此漏洞。定期检查并应用安全更新。
2. **安全配置**：在构建服务器上实施严格的安全配置，限制对敏感文件的访问权限。
3. **代码审查**：定期进行代码审查，特别是对 package.json 文件中的 postinstall 脚本进行审查，以防止恶意代码注入。
4. **监控和日志记录**：启用构建服务器的监控和日志记录，以便在发生异常活动时能够及时检测和响应。

通过以上措施，可以有效降低 CVE-2025-27554 漏洞带来的安全风险。